In [1]:
import sys; sys.path.insert(0,'/Users/ryan/github/cadence/')
import sys; sys.path.insert(0,'/Users/ryan/github/lltk/')
from cadence.imports import *

In [2]:
# s="""Doth teach    \t  that ease's\nand that repose,    to say 'Thus far the miles are measured from thy friend!'"""
s='How do you know \n  dude?'

In [8]:
parse(s)

Mapping do_parse_iter() [x1]:   0%|                                                  | 0/1 [00:00<?, ?it/s]

from cache: ['How do you know  \n   dude? ____parse____*f-res|*s/unstressed|*w-res|*w/peak|*w/stressed']


KeyError: "None of [Index(['word_i', 'syll_i', 'word_ipa_i'], dtype='object')] are in the [columns]"

In [3]:
# parse(s)

In [4]:
def get_line_data(linepart_txt,**kwargs):
    try:
        df=line2df(linepart_txt,**kwargs)
        df['word_i']=df['word_i'].rank(method='dense').apply(int)
        df=df[df.syll_i!=0]
        # df['linepart_str']=linepart_txt
        df['linepart_num_syll']=len(df[df['word_ipa_i']==1])
        df['linepart_num_monosyll']=sum([
            1 if len(g)==1 else 0
            for i,g in df[df.word_ipa_i==1].groupby('word_i')
        ])
        assign_proms(df)
        # df=setindex(df)
        return df
    except Exception as e:
        return pd.DataFrame()


In [40]:
def get_scansion(linepart_txt,path=None,cache=True,force=False,**kwargs):
    if type(linepart_txt)==pd.DataFrame:
        return linepart_txt
    
    # lpl=linepart_txt.strip().split()
    #lptxt=' '.join(lpl)
    #lpkey='_'.join(lpl)
    lptxt=linepart_txt#.strip().replace(' ','_')
    lpkey=lptxt
    key=f'{lpkey}{DBSEP}scan'
    if not force and cache:
        with get_db(path=path) as d:
            if not key in d:
                d[key]=get_line_data(lptxt,**kwargs)
            odf=d[key]
    else:
        odf=get_line_data(lptxt)
    
    if len(odf): odf['linepart_str']=lptxt
    return setindex(odf)

In [44]:
get_scansion(s)

is_funcword  ...  prom_weight
linepart_str              word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight               ...             
How do you know \n  dude? 1      How      1          haʊ      1      How      haʊ      U                                  1  ...          NaN
                          2      do       1          'duː     1      do       'duː     P                                  0  ...          NaN
                                          2          duː      1      do       duː      U                                  1  ...          NaN
                          3      you      1          juː      1      you      juː      U                                  1  ...          NaN
                          4      know     1          'noʊ     1      know     'noʊ     P                                  0  ...          NaN
                          5      dude?    1          'duːd    1      dude?    'duːd    P                                  0  ...          NaN

[6 rows x 12 columns]

In [46]:
lineparts(s)

,stanza_i,line_i,linepart_i,linepart_str
0,1,1,1,How do you know \n dude?


In [52]:
def scan(txt_or_fn,**kwargs):
    o=list(scan_iter(txt_or_fn,**kwargs))
    return pd.concat(o) if len(o) else pd.DataFrame()

def scan_iter(df_or_txt_or_fn,num_proc=1,lim=None,progress=True,**kwargs):
    txt=to_txt(df_or_txt_or_fn)
    df=lineparts(txt,**kwargs)
    rows=df.to_dict(orient='records')
    iterr_o=pmap_iter(
        do_scan_iter,
        rows,
        progress=progress,
        num_proc=num_proc,
        desc='Scanning lines'
    )
    for i,odf in enumerate(iterr_o):
        if lim and i>=lim: break
        yield odf

def do_scan_iter(rowd,**kwargs):
    lpstr=rowd['linepart_str']
    del rowd['linepart_str']
    odf=get_scansion(lpstr,**kwargs)
    for k,v in rowd.items(): odf[k]=v
    return setindex(odf)


In [57]:
scan(s,verse=True)

Scanning lines [x1]: 100%|███████████████████████████████████████████████████| 2/2 [00:00<00:00, 48.53it/s]


is_funcword  ...  prom_weight
stanza_i line_i linepart_i linepart_str    word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight               ...             
1        1      1          How do you know 1      How      1          haʊ      1      How      haʊ      U                                  1  ...          NaN
                                           2      do       1          'duː     1      do       'duː     P                                  0  ...          NaN
                                                           2          duː      1      do       duː      U                                  1  ...          NaN
                                           3      you      1          juː      1      you      juː      U                                  1  ...          NaN
                                           4      know     1          'noʊ     1      know     'noʊ     P                                  0  ...          NaN
         2      1          dude?           1      dude?    1          'duːd    1      dude?    'duːd    P                                  0  ...          NaN

[6 rows x 12 columns]

In [58]:

# def get_scansion_key(scansion_df_or_str,**kwargs):
#     sdf=get_scansion(scansion_df_or_str,**kwargs)
    
#     # sdf
#     o=[]
#     for word_i,wdf in sdf.groupby('word_i'):
#         x=[]
#         for word_ipa_i,widf in wdf.groupby('word_ipa_i'):
#             x+=[widf.word_ipa.iloc[0]]
#         o+=['/'.join(x)]
#     ostr='_'.join(o)
#     return ostr

def get_parse_cache(linepart_str,path=None,**kwargs):
    #lpdf=get_scansion(linepart_str,**kwargs)
    lpdf_key=linepart_str#get_scansion_key(lpdf)
    cs="|".join(sorted(list(kwargs.get("constraints",DEFAULT_CONSTRAINTS))))
    dbk=f'{lpdf_key}{DBSEP}parse{DBSEP}{cs}'
    with get_db(path=path) as db:
        if not dbk in db:
            db[dbk]=get_parse_nocache(linepart_str,**kwargs)
        else:
            print('from cache:',[dbk])
        return db[dbk]

def get_parse_nocache(linepart_str,path=None,**kwargs):
    print('no cache:',[linepart_str])
    lpdf=get_scansion(linepart_str,**kwargs)
    if len(lpdf):
        pdf=parse_linepartdf(lpdf,**kwargs)
        # strcols={'word_str', 'syll_str', 'linepart_str'}
        if len(pdf):
            # pdf=pdf[[col for col in pdf.columns if col not in strcols]]
            return pdf
    return pd.DataFrame()

def get_parse(linepart_str_or_df,path=None,cache=True,force=False,**kwargs):
    linepart_str = get_linepart_str(linepart_str_or_df)
    linepart_df = get_linepart_df(linepart_str_or_df)
        
    if cache and not force:
        pdf=get_parse_cache(linepart_str,path=path,**kwargs)
    else:
        pdf=get_parse_nocache(linepart_str,path=path,**kwargs)
            
    cols1=set(linepart_df.columns)
    cols2=set(pdf.columns)
    row=linepart_df.iloc[0]
    joiner=['word_i','word_ipa_i','syll_i']
    odf=pdf.merge(linepart_df[(cols1-cols2) | set(joiner)],on=joiner,how='left')
    if len(odf): odf['linepart_str']=linepart_str
    return setindex(odf)

    
def parse_linepartdf(linepartdf,linepart_str=None,**kwargs):
    odf=resetindex(linepartdf)
    odf=parse_prosodic_line(odf,**kwargs)
    odf=postproc_dfline(odf,**kwargs)
    if 'index' in odf.columns: odf=odf.drop('index',1)
    return odf


def do_parse_iter(obj):
    lpstr,metad,attrs = obj
    odf=get_parse(lpstr,**attrs)
    for k,v in metad.items(): odf[k]=v
    return setindex(odf)


def parse_iter(
        txt_or_txtdf,
        num_proc=DEFAULT_NUM_PROC,
        progress=True,
        lim=None,
        shuffle=False,
        **kwargs
    ):
    try:
        # Get simple line parts
        lpdf = lineparts(txt_or_txtdf, **kwargs)
        # set up
        objs = [
            (astr, dict(stanza_i=x,line_i=y,linepart_i=z), kwargs)
            for (x,y,z,astr) in zip(
                lpdf.stanza_i,
                lpdf.line_i,
                lpdf.linepart_i,
                lpdf.linepart_str
            )
        ]
        if shuffle: random.shuffle(objs)
        if lim: objs=objs[:lim]
        # print(objs)
        
        proc_gen=pmap_iter(do_parse_iter, objs, num_proc=num_proc, progress=progress)
        yield from proc_gen
    except AssertionError:
    # except (KeyError,AttributeError) as e:
        # print('!!',e)
        yield pd.DataFrame()

def parse(txt_or_txtdf,*x,verbose=False,**y):
    """
    Return parses
    """
    o=list(parse_iter(txt_or_txtdf,*x, verbose=verbose, **y))
    odf=pd.concat(o) if len(o) else pd.DataFrame()
    return odf


In [59]:
get_parse(s)

from cache: ['How do you know \n  dude?____parse____*f-res|*s/unstressed|*w-res|*w/peak|*w/stressed']


*total  ...  prom_weight
linepart_str              parse_rank is_troch parse_i parse parse_str             combo_stress combo_ipa               combo_i parse_is_bounded parse_bounded_by parse_pos_i parse_pos word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
How do you know \n  dude? 1          0        1       wswsw how|DO|you|KNOW|dude* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                 2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                 3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                 4           s         4      know     1          'noʊ     1      know     'noʊ     P                       1            s              0.0  ...          NaN
                                                                                                                                                                 5           w         5      dude?    1          'duːd    1      dude?    'duːd    P                       1            w              1.0  ...          NaN
                          2          0        2       wswss how|DO|you|KNOW.DUDE* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                 2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                 3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                 4           ss        4      know     1          'noʊ     1      know     'noʊ     P                       1            s              1.0  ...          NaN
                                                                                                                                                                                       5      dude?    1          'duːd    1      dude?    'duːd    P                       2            s              1.0  ...          NaN

[10 rows x 23 columns]

In [60]:
for x in parse_iter(s,force=True): break
x

Mapping do_parse_iter() [x1]:   0%|                                                  | 0/1 [00:00<?, ?it/s]

no cache: ['How do you know  \n   dude? ']


*total  ...  prom_weight
stanza_i line_i linepart_i linepart_str                 parse_rank is_troch parse_i parse parse_str             combo_stress combo_ipa               combo_i parse_is_bounded parse_bounded_by parse_pos_i parse_pos word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
1        1      1          How do you know  \n   dude?  1          0        1       wswsw how|DO|you|KNOW|dude* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               4           s         4      know     1          'noʊ     1      know     'noʊ     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               5           w         11     dude?    1          'duːd    1      dude?    'duːd    P                       1            w              1.0  ...          NaN
                                                        2          0        2       wswss how|DO|you|KNOW.DUDE* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               4           ss        4      know     1          'noʊ     1      know     'noʊ     P                       1            s              1.0  ...          NaN
                                                                                                                                                                                                                     11     dude?    1          'duːd    1      dude?    'duːd    P                       2            s              1.0  ...          NaN

[10 rows x 23 columns]

In [63]:
for x in parse_iter(s,force=False,verse=True): break
x

Mapping do_parse_iter() [x1]:   0%|                                                  | 0/2 [00:00<?, ?it/s]

from cache: ['How do you know____parse____*f-res|*s/unstressed|*w-res|*w/peak|*w/stressed']


*total  ...  prom_weight
stanza_i line_i linepart_i linepart_str    parse_rank is_troch parse_i parse parse_str       combo_stress combo_ipa         combo_i parse_is_bounded parse_bounded_by parse_pos_i parse_pos word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
1        1      1          How do you know 1          0        1       wsws  how|DO|you|KNOW U P U P      haʊ 'duː juː 'noʊ 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                      2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                      3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                      4           s         4      know     1          'noʊ     1      know     'noʊ     P                       1            s              0.0  ...          NaN

[4 rows x 23 columns]

In [55]:
# setindex(get_parse(s))

In [47]:
get_parse(s)

from cache: haʊ_'duː/duː_juː_'noʊ_'duːd


,word_i,word_ipa_i,syll_i,word_ipa,syll_ipa,is_funcword,linepart_num_syll,linepart_num_monosyll,prom_stress,prom_weight,syll_stress,syll_weight,prom_strength,is_stressed,is_unstressed,is_heavy,is_light,is_peak,is_trough,parse_i,parse,parse_str,parse_pos_i,parse_pos,parse_syll_i,parse_syll,*f-res,*s/unstressed,*w-res,*w/peak,*w/stressed,*total,is_w,is_s,parse_num_pos,parse_num_syll,parse_is_bounded,parse_bounded_by,combo_i,combo_stress,combo_ipa,is_troch,parse_rank,num_parses,word_str,syll_str,linepart_str
0,1,1,1,haʊ,haʊ,1,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,1,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2,How,How,How do you know \n dude?
1,2,1,1,'duː,'duː,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,2,s,1,s,0.0,0.0,0,0.0,0.0,0.0,0,1,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2,do,do,How do you know \n dude?
2,3,1,1,juː,juː,1,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,3,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2,you,you,How do you know \n dude?
3,4,1,1,'noʊ,'noʊ,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,4,s,1,s,0.0,0.0,0,0.0,0.0,0.0,0,1,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2,know,know,How do you know \n dude?
4,5,1,1,'duːd,'duːd,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,5,w,1,w,0.0,0.0,0,0.0,1.0,1.0,1,0,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2,dude?,dude?,How do you know \n dude?
5,1,1,1,haʊ,haʊ,1,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,1,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2,How,How,How do you know \n dude?
6,2,1,1,'duː,'duː,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,2,s,1,s,0.0,0.0,0,0.0,0.0,0.0,0,1,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2,do,do,How do you know \n dude?
7,3,1,1,juː,juː,1,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,3,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2,you,you,How do you know \n dude?
8,4,1,1,'noʊ,'noʊ,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,4,ss,1,s,1.0,0.0,0,0.0,0.0,1.0,0,1,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2,know,know,How do you know \n dude?
9,5,1,1,'duːd,'duːd,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,4,ss,2,s,1.0,0.0,0,0.0,0.0,1.0,0,1,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2,dude?,dude?,How do you know \n dude?


In [35]:
parse(s,force=True)

Mapping do_parse_iter() [x1]: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 18.34it/s]

How do you know  \n   dude?     10
Name: linepart_str, dtype: int64


*total  ...  prom_weight
stanza_i line_i linepart_i linepart_str                 parse_rank is_troch parse_i parse parse_str             combo_stress combo_ipa               combo_i parse_is_bounded parse_bounded_by parse_pos_i parse_pos word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
1        1      1          How do you know  \n   dude?  1          0        1       wswsw how|DO|you|KNOW|dude* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               4           s         4      know     1          'noʊ     1      know     'noʊ     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               5           w         11     dude?    1          'duːd    1      dude?    'duːd    P                       1            w              1.0  ...          NaN
                                                        2          0        2       wswss how|DO|you|KNOW.DUDE* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               4           ss        4      know     1          'noʊ     1      know     'noʊ     P                       1            s              1.0  ...          NaN
                                                                                                                                                                                                                     11     dude?    1          'duːd    1      dude?    'duːd    P                       2            s              1.0  ...          NaN

[10 rows x 23 columns]

In [36]:
parse(s,force=False)

Mapping do_parse_iter() [x1]: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 28.25it/s]

How do you know  \n   dude?     8
Name: linepart_str, dtype: int64


*total  ...  prom_weight
stanza_i line_i linepart_i linepart_str                 parse_rank is_troch parse_i parse parse_str             combo_stress combo_ipa               combo_i parse_is_bounded parse_bounded_by parse_pos_i parse_pos word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
1        1      1          How do you know  \n   dude?  1          0        1       wswsw how|DO|you|KNOW|dude* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               4           s         4      know     1          'noʊ     1      know     'noʊ     P                       1            s              0.0  ...          NaN
                                                        2          0        2       wswss how|DO|you|KNOW.DUDE* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                                               3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                                               4           ss        4      know     1          'noʊ     1      know     'noʊ     P                       1            s              1.0  ...          NaN
                           NaN                          1          0        1       wswsw how|DO|you|KNOW|dude* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             5           w         5      NaN      1          'duːd    1      NaN      'duːd    P                       1            w              1.0  ...          NaN
                                                        2          0        2       wswss how|DO|you|KNOW.DUDE* U P U P P    haʊ 'duː juː 'noʊ 'duːd 1       False                             4           ss        5      NaN      1          'duːd    1      NaN      'duːd    P                       2            s              1.0  ...          NaN

[10 rows x 23 columns]

In [22]:
get_scansion(s)

,word_i,word_ipa_i,syll_i,word_str,word_ipa,syll_ipa,syll_str,is_funcword,linepart_num_syll,linepart_num_monosyll,prom_stress,prom_weight,syll_stress,syll_weight,prom_strength,is_stressed,is_unstressed,is_heavy,is_light,is_peak,is_trough,linepart_str
0,1,1,1,How,haʊ,haʊ,How,1,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,How do you know \n dude?
1,2,1,1,do,'duː,'duː,do,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,How do you know \n dude?
2,2,2,1,do,duː,duː,do,1,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,How do you know \n dude?
3,3,1,1,you,juː,juː,you,1,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,How do you know \n dude?
4,4,1,1,know,'noʊ,'noʊ,know,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,How do you know \n dude?
5,5,1,1,dude?,'duːd,'duːd,dude?,0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,How do you know \n dude?


In [23]:
df=setindex(get_scansion(s))
set(df.index.names)

{'linepart_str',
 'syll_i',
 'syll_ipa',
 'syll_str',
 'syll_stress',
 'syll_weight',
 'word_i',
 'word_ipa',
 'word_ipa_i',
 'word_str'}

In [24]:
scan(s)

Scanning lines [x1]: 100%|███████████████████████████████████████████████████| 1/1 [00:00<00:00, 60.53it/s]


is_funcword  ...  prom_weight
stanza_i line_i linepart_i linepart_str                word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight               ...             
1        1      1          How do you know  \n   dude? 1      How      1          haʊ      1      How      haʊ      U                                  1  ...          NaN
                                                       2      do       1          'duː     1      do       'duː     P                                  0  ...          NaN
                                                                       2          duː      1      do       duː      U                                  1  ...          NaN
                                                       3      you      1          juː      1      you      juː      U                                  1  ...          NaN
                                                       4      know     1          'noʊ     1      know     'noʊ     P                                  0  ...          NaN
                                                       11     dude?    1          'duːd    1      dude?    'duːd    P                                  0  ...          NaN

[6 rows x 12 columns]

In [25]:
parse(s,force=True,linebreaks=True)

Mapping do_parse_iter() [x1]: 100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 24.14it/s]


*total  ...  prom_weight
stanza_i line_i linepart_i linepart_str     parse_rank is_troch parse_i parse parse_str       combo_stress combo_ipa         combo_i parse_is_bounded parse_bounded_by parse_pos_i parse_pos word_i word_str word_ipa_i word_ipa syll_i syll_str syll_ipa syll_stress syll_weight parse_syll_i parse_syll          ...             
1        1      1          How do you know  1          0        1       wsws  how|DO|you|KNOW U P U P      haʊ 'duː juː 'noʊ 1       False                             1           w         1      How      1          haʊ      1      How      haʊ      U                       1            w              0.0  ...          NaN
                                                                                                                                                                       2           s         2      do       1          'duː     1      do       'duː     P                       1            s              0.0  ...          NaN
                                                                                                                                                                       3           w         3      you      1          juː      1      you      juː      U                       1            w              0.0  ...          NaN
                                                                                                                                                                       4           s         4      know     1          'noʊ     1      know     'noʊ     P                       1            s              0.0  ...          NaN
         2      1          dude?            1          1        1       s     DUDE            P            'duːd             1       False                             1           s         1      dude?    1          'duːd    1      dude?    'duːd    P                       1            s              0.0  ...          NaN
                                            2          0        2       w     dude*           P            'duːd             1       False                             1           w         1      dude?    1          'duːd    1      dude?    'duːd    P                       1            w              1.0  ...          NaN

[6 rows x 23 columns]

In [29]:
df=get_parse(s)
df

from cache: How do you know 
  dude?


,syll_str,word_i,syll_i,linepart_str,word_ipa_i,word_str,word_ipa,syll_ipa,is_funcword,linepart_num_syll,linepart_num_monosyll,prom_stress,prom_weight,syll_stress,syll_weight,prom_strength,is_stressed,is_unstressed,is_heavy,is_light,is_peak,is_trough,parse_i,parse,parse_str,parse_pos_i,parse_pos,parse_syll_i,parse_syll,*f-res,*s/unstressed,*w-res,*w/peak,*w/stressed,*total,is_w,is_s,parse_num_pos,parse_num_syll,parse_is_bounded,parse_bounded_by,combo_i,combo_stress,combo_ipa,is_troch,parse_rank,num_parses
0,How,1,1,How do you know \n dude?,1,How,haʊ,haʊ,1.0,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,1,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2
1,do,2,1,How do you know \n dude?,1,do,'duː,'duː,0.0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,2,s,1,s,0.0,0.0,0,0.0,0.0,0.0,0,1,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2
2,you,3,1,How do you know \n dude?,1,you,juː,juː,1.0,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,3,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2
3,know,4,1,How do you know \n dude?,1,know,'noʊ,'noʊ,0.0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,4,s,1,s,0.0,0.0,0,0.0,0.0,0.0,0,1,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2
4,NaN,11,1,NaN,1,NaN,'duːd,'duːd,0.0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,1,wswsw,how|DO|you|KNOW|dude*,5,w,1,w,0.0,0.0,0,0.0,1.0,1.0,1,0,5,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,1,2
5,How,1,1,How do you know \n dude?,1,How,haʊ,haʊ,1.0,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,1,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2
6,do,2,1,How do you know \n dude?,1,do,'duː,'duː,0.0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,2,s,1,s,0.0,0.0,0,0.0,0.0,0.0,0,1,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2
7,you,3,1,How do you know \n dude?,1,you,juː,juː,1.0,5,5,0.0,NaN,U,,NaN,0,1,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,3,w,1,w,0.0,0.0,0,0.0,0.0,0.0,1,0,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2
8,know,4,1,How do you know \n dude?,1,know,'noʊ,'noʊ,0.0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,4,ss,1,s,1.0,0.0,0,0.0,0.0,1.0,0,1,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2
9,NaN,11,1,NaN,1,NaN,'duːd,'duːd,0.0,5,5,1.0,NaN,P,,NaN,1,0,0,0,0,0,2,wswss,how|DO|you|KNOW.DUDE*,4,ss,2,s,1.0,0.0,0,0.0,0.0,1.0,0,1,4,5,False,,1,U P U P P,haʊ 'duː juː 'noʊ 'duːd,0,2,2


In [16]:
for sx in [s, s.lower(), s.upper(), s.title()]:
    sdf=setindex(get_parse(sx,force=False))
    #display(sdf)

from cache: How do you know 
  dude?
from cache: how do you know 
  dude?
from cache: HOW DO YOU KNOW 
  DUDE?
from cache: How Do You Know 
  Dude?


In [10]:
# get_scansion(s)

In [12]:
get_parse(s)

dɔːθ_'tiːʧ_'ðæt/ðət_'iː.zɪz_ænd_'ðæt/ðət_riː.'poʊz_tuː_'seɪ_'ðʌs_'fɑr_ðə_'maɪlz/'maɪ.əlz_ɑr_'mɛ.ʒɛːd_frʌm_ðaɪ_'frɛnd____parse____*f-res|*s/unstressed|*w-res|*w/peak|*w/stressed


,word_i,word_ipa_i,syll_i,word_str,word_ipa,syll_ipa,syll_str,is_funcword,linepart_num_syll,linepart_num_monosyll,prom_stress,prom_weight,syll_stress,syll_weight,prom_strength,is_stressed,is_unstressed,is_heavy,is_light,is_peak,is_trough,linepart_str,parse_i,parse,parse_str,parse_pos_i,parse_pos,parse_syll_i,parse_syll,*f-res,*s/unstressed,*w-res,*w/peak,*w/stressed,*total,is_w,is_s,parse_num_pos,parse_num_syll,parse_is_bounded,parse_bounded_by,combo_i,combo_stress,combo_ipa,is_troch,parse_rank,num_parses
0,1,1,1,Doth,dɔːθ,dɔːθ,Doth,0.0,21,15,0.0,NaN,U,,NaN,0,1,0,0,0,0,Doth teach \t that ease's\nand that repose...,1,wswswwswswswswswswsws,doth|TEACH|that|EA|ses.and*|THAT|re|POSE|to|SA...,1,w,1,w,0.0,0.0,0.0,0.0,0.0,0.0,1,0,20,21,False,,1,U P U PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ ðət 'iː.zɪz ænd 'ðæt riː.'poʊz tuː ...,0,1,6
1,2,1,1,teach,'tiːʧ,'tiːʧ,teach,0.0,21,15,1.0,NaN,P,,NaN,1,0,0,0,0,0,Doth teach \t that ease's\nand that repose...,1,wswswwswswswswswswsws,doth|TEACH|that|EA|ses.and*|THAT|re|POSE|to|SA...,2,s,1,s,0.0,0.0,0.0,0.0,0.0,0.0,0,1,20,21,False,,1,U P U PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ ðət 'iː.zɪz ænd 'ðæt riː.'poʊz tuː ...,0,1,6
2,10,2,1,that,ðət,ðət,that,1.0,21,15,0.0,NaN,U,,NaN,0,1,0,0,0,0,Doth teach \t that ease's\nand that repose...,1,wswswwswswswswswswsws,doth|TEACH|that|EA|ses.and*|THAT|re|POSE|to|SA...,3,w,1,w,0.0,0.0,0.0,0.0,0.0,0.0,1,0,20,21,False,,1,U P U PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ ðət 'iː.zɪz ænd 'ðæt riː.'poʊz tuː ...,0,1,6
3,11,1,1,ease's,'iː.zɪz,'iː,ea,0.0,21,15,1.0,NaN,P,,1.0,1,0,0,0,1,0,Doth teach \t that ease's\nand that repose...,1,wswswwswswswswswswsws,doth|TEACH|that|EA|ses.and*|THAT|re|POSE|to|SA...,4,s,1,s,0.0,0.0,0.0,0.0,0.0,0.0,0,1,20,21,False,,1,U P U PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ ðət 'iː.zɪz ænd 'ðæt riː.'poʊz tuː ...,0,1,6
4,11,1,2,ease's,'iː.zɪz,zɪz,se's,0.0,21,15,0.0,NaN,U,,0.0,0,1,0,0,0,1,Doth teach \t that ease's\nand that repose...,1,wswswwswswswswswswsws,doth|TEACH|that|EA|ses.and*|THAT|re|POSE|to|SA...,5,ww,1,w,1.0,0.0,0.0,0.0,0.0,1.0,1,0,20,21,False,,1,U P U PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ ðət 'iː.zɪz ænd 'ðæt riː.'poʊz tuː ...,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,26,1,1,measured,'mɛ.ʒɛːd,'mɛ,mea,0.0,21,15,1.0,NaN,P,,1.0,1,0,0,0,1,0,Doth teach \t that ease's\nand that repose...,6,swswswswswswswswsswws,DOTH*|teach*|THAT|ea*|SES*|and|THAT|re|POSE|to...,17,ss,1,s,0.0,1.0,0.0,0.0,0.0,1.0,0,1,19,21,False,,2,U P P PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ 'ðæt 'iː.zɪz ænd 'ðæt riː.'poʊz tuː...,1,6,6
17,26,1,2,measured,'mɛ.ʒɛːd,ʒɛːd,sured,0.0,21,15,0.0,NaN,U,,0.0,0,1,0,0,0,1,Doth teach \t that ease's\nand that repose...,6,swswswswswswswswsswws,DOTH*|teach*|THAT|ea*|SES*|and|THAT|re|POSE|to...,17,ss,2,s,0.0,1.0,0.0,0.0,0.0,1.0,0,1,19,21,False,,2,U P P PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ 'ðæt 'iː.zɪz ænd 'ðæt riː.'poʊz tuː...,1,6,6
18,27,1,1,from,frʌm,frʌm,from,1.0,21,15,0.0,NaN,U,,NaN,0,1,0,0,0,0,Doth teach \t that ease's\nand that repose...,6,swswswswswswswswsswws,DOTH*|teach*|THAT|ea*|SES*|and|THAT|re|POSE|to...,18,ww,1,w,0.0,0.0,0.0,0.0,0.0,0.0,1,0,19,21,False,,2,U P P PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ 'ðæt 'iː.zɪz ænd 'ðæt riː.'poʊz tuː...,1,6,6
19,28,1,1,thy,ðaɪ,ðaɪ,thy,1.0,21,15,0.0,NaN,U,,NaN,0,1,0,0,0,0,Doth teach \t that ease's\nand that repose...,6,swswswswswswswswsswws,DOTH*|teach*|THAT|ea*|SES*|and|THAT|re|POSE|to...,18,ww,2,w,0.0,0.0,0.0,0.0,0.0,0.0,1,0,19,21,False,,2,U P P PU U P UP U P P P U P U PU U U P,dɔːθ 'tiːʧ 'ðæt 'iː.zɪz ænd 'ðæt riː.'poʊz tuː...,1,6,6


In [2]:
import lltk
C=lltk.load('Saintsbury')
txt=C.au.Browne.ReligioMedici.txt.replace('\n\n','\n')#[:2000]
s=txt.split('\n')[0]
s

'A true and full copy of it which was most'

In [3]:
lpdf=scan(s)
# lpdf

Scanning lines [x1]: 100%|██████████████████████████████████████████████████| 1/1 [00:00<00:00, 314.37it/s]


In [10]:
# def get_parse(linepart_str_or_df,path=None,cache=True,force=False,**kwargs):
#     linepart_str = get_linepart_str(linepart_str_or_df)
#     linepart_df = get_linepart_df(linepart_str_or_df)
        
#     if cache and not force:
#         pdf=get_parse_cache(linepart_str,path=path,**kwargs)
#     else:
#         pdf=get_parse_nocache(linepart_str,path=path,**kwargs)
    
#     cols1=set(linepart_df.columns)
#     cols2=set(pdf.columns)
#     row=linepart_df.iloc[0]
#     for col in cols1-cols2: pdf[col]=row[col]
#     pdf['linepart_str']=linepart_str
#     return pdf
# #     

In [4]:
%%timeit
parse(txt,lim=10,force=True)

Mapping do_parse_iter() [x1]: 100%|████████████████████████████████████████| 10/10 [00:00<00:00, 11.30it/s]

1.61 s ± 21.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
parse(txt,lim=10,force=False)

Mapping do_parse_iter() [x1]: 100%|████████████████████████████████████████| 10/10 [00:00<00:00, 71.42it/s]

840 ms ± 6.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
stop

In [ ]:
%%timeit
get_parse(s)

In [ ]:
%%timeit
get_parse_nocache(s)

In [ ]:
# rank_parses??

In [ ]:
# rank_parses(get_parse(s))

In [ ]:
odf=setindex(get_parse(s))
# odf

In [ ]:
# def load_parses(dfparses_or_path):
#     if type(dfparses_or_path)==str:
#         if os.path.exists(dfparses_or_path):
#             dfparses=read_df(dfparses_or_path)
#         else:
#             return pd.DataFrame()
#     else:
#         dfparses=dfparses_or_path
#     return dfparses
 
# def info_parses(dfparses_or_path,lcols=['stanza_i','line_i','linepart_i'],min_num_syll=4):
#     infod={}
#     dfparses=load_parses(dfparses_or_path)
    

#     # byline
#     allcols=set(dfparses.columns)|set(dfparses.index.names)
#     if 'syll_i' in allcols: dfparses=to_lines(dfparses)
#     allcols=set(dfparses.columns)|set(dfparses.index.names)
#     if 'parse_num_syll' not in allcols: return infod
#     dfparses=dfparses.query(f'parse_num_syll>={min_num_syll}')


#     infod['num_lines_total']=dfparses.groupby(['stanza_i','line_i','linepart_i']).size().shape[0]

#     # avg num sylls
#     num_sylls = dfparses.groupby(lcols).parse_num_syll.mean()
#     num_monosylls = dfparses.groupby(lcols).linepart_num_monosyll.mean()
#     infod['num_sylls_avg'] = num_sylls.mean()

#     infod['num_monosylls_avg'] = num_monosylls.mean()
#     num_monosylls_per_10syll = num_monosylls / num_sylls * 10
#     infod['num_monosylls_per_10syll_avg'] = num_monosylls_per_10syll.mean()


#     # avg num parses per line?
#     num_parses = dfparses.groupby(lcols).size()# / dfparses.groupby(lcols).parse_num_syll.mean()
#     infod['num_parses_avg'] = num_parses.mean()
#     num_parses_per_10syll = num_parses / num_sylls * 10
#     infod['num_parses_per_10syll_avg'] = num_parses_per_10syll.mean()


#     # avg viols?
#     vkeys=[col for col in dfparses.columns if col.startswith('*')]
#     for vk in vkeys:
#         vk2='' if vk=='*total' else f'__{vk}'
#         num_viols = dfparses.groupby(lcols)[vk].mean()
#         infod[f'num_viols_avg{vk2}'] = num_viols.mean()
#         num_viols_per_10syll = num_viols /num_sylls * 10
#         infod[f'num_viols_per_10syll_avg{vk2}'] = num_viols_per_10syll.mean()

#     return infod
#     except KeyError:
#         return infod
    

In [ ]:
odf=to_lines(odf)[['num_parses']]
odf

In [ ]:
stop

In [ ]:
odf.linepart_str

In [ ]:
get_parse(s)

In [ ]:
%%timeit
to_lineparts(txt)

In [ ]:
%%timeit
lineparts(txt)

In [ ]:
parse(s)